In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

root_path = 'S:\PCRC 186 Lou'
data_path = 'S:\PCRC 186 Lou\Tab delimited'

### Race info (aggregated and institution-wise) for caseinfo2

In [69]:
def race_prevalence_hospital(df):

    #overall_prevalence = df['Race'].value_counts(normalize=True) * 100
    race_agg = pd.DataFrame(df['Race'].value_counts()).reset_index(drop = False).rename(columns = {'Race':'count', 'index':'race'})
    race_agg['count%'] = (race_agg['count']/race_agg['count'].sum())*100
    assert race_agg['count%'].sum() == 100

    institution_prevalence = pd.pivot_table(df, index='Institution', columns='Race', aggfunc='size', fill_value=0)

    institution_prevalence_percentage = institution_prevalence.div(institution_prevalence.sum(axis=1), axis=0) * 100

    inst_race_df = pd.concat([institution_prevalence, institution_prevalence_percentage], axis=1, keys=[ 'count', 'count%'])

    return race_agg, inst_race_df

#Compute the prevalence of each race category across the entirety of caseinfo2 and at each individual 
#hospital (i.e., make a table with one row for each institution, and columns for the
#and % of each race category at that institution).

#caseinfo2 = pd.read_csv(os.path.join(data_path, 'PCRC_186_SDF_CaseInfo2_20230309.csv')).drop_duplicates()

bad_inst_list = [67, 118, 92, 101, 91, 115, 102, 99, 13, 98]
caseinfo2_filtered = caseinfo2.loc[~caseinfo2.Institution.isin(bad_inst_list)].reset_index(drop = True)

race_agg, inst_race_df = race_prevalence_hospital(caseinfo2_filtered)

save_dir = 'S:/PCRC 186 Lou/Sayantan/MPOG_work/Results'
race_agg.to_csv(os.path.join(save_dir, "race_agg_caseinfo2.csv"), index=False)
inst_race_df.to_csv(os.path.join(save_dir, "inst_race_caseinfo2.csv"), index=True)

In [70]:
#---------Loading caseinfo and cpt files -------------

caseinfo1 = pd.read_csv(os.path.join(data_path, 'PCRC_186_SDF_CaseInfo1_20230309.csv')).drop_duplicates()
caseinfo2 = pd.read_csv(os.path.join(data_path, 'PCRC_186_SDF_CaseInfo2_20230309.csv')).drop_duplicates()

caseinfo1['MPOG_Patient_ID'] = caseinfo1['MPOG_Patient_ID'].str.replace('-','').str.lower()
caseinfo1['MPOG_Case_ID'] = caseinfo1['MPOG_Case_ID'].str.replace('-','').str.lower()

caseinfo2['MPOG_Patient_ID'] = caseinfo2['MPOG_Patient_ID'].str.replace('-','').str.lower()
caseinfo2['MPOG_Case_ID'] = caseinfo2['MPOG_Case_ID'].str.replace('-','').str.lower()

#------------- Converting the units of height and weight ----------

caseinfo1['Height (inch)'] = caseinfo1['Height (cm)']* 0.4
caseinfo1['Weight (pounds)'] = caseinfo1['Weight (kg)']* 2.2
caseinfo2['Height (inch)'] = caseinfo2['Height (cm)']* 0.4
caseinfo2['Weight (pounds)'] = caseinfo2['Weight (kg)']* 2.2

#------------- Creating labels (transfusion needed or not) ------
caseinfo1['NOTHBLEED'] = None
caseinfo1.loc[caseinfo1['Blood Product Total - PRBCs (ml)'] == 0, 'NOTHBLEED'] = 0
caseinfo1.loc[caseinfo1['Blood Product Total - PRBCs (ml)'] > 0, 'NOTHBLEED'] = 1

caseinfo2['NOTHBLEED'] = None
caseinfo2.loc[caseinfo2['Blood Product Total - PRBCs (ml)'] == 0, 'NOTHBLEED'] = 0
caseinfo2.loc[caseinfo2['Blood Product Total - PRBCs (ml)'] > 0, 'NOTHBLEED'] = 1

print('Number of entries in Caseinfo1 = {}'.format(len(caseinfo1)))
print('Number of entries in Caseinfo2 = {}'.format(len(caseinfo2)))

print('Number of unique caseID in Caseinfo1 = {}'.format(len(caseinfo1.MPOG_Case_ID.unique())))
print('Number of unique caseID in Caseinfo2 = {}'.format(len(caseinfo2.MPOG_Case_ID.unique())))


Number of entries in Caseinfo1 = 7060060
Number of entries in Caseinfo2 = 4155859
Number of unique caseID in Caseinfo1 = 7060060
Number of unique caseID in Caseinfo2 = 4155859


### Add smoke info

In [71]:
#------------ Adding smoke information from addinfo (current user/former user) ----------

addinfo1 = pd.read_csv(os.path.join(data_path, 'PCRC_186_AddtlPhenotypes1.csv')).drop_duplicates()
addinfo2 = pd.read_csv(os.path.join(data_path, 'PCRC_186_AddtlPhenotypes2.csv')).drop_duplicates()

addinfo1['SMOKE'] = 0
addinfo1.loc[addinfo1['SmokingTobaccoClassification'].isin(['Current User']), 'SMOKE'] = 1
addinfo1['MPOG_Patient_ID'] = addinfo1['MPOG_Patient_ID'].str.replace('-','').str.replace('}','').str.replace('{','').str.lower()
addinfo1['MPOG_Case_ID'] = addinfo1['MPOG_Case_ID'].str.replace('-','').str.replace('}','').str.replace('{','').str.lower()

addinfo2['SMOKE'] = 0
addinfo2.loc[addinfo2['SmokingTobaccoClassification'].isin(['Current User']), 'SMOKE'] = 1
addinfo2['MPOG_Patient_ID'] = addinfo2['MPOG_Patient_ID'].str.replace('-','').str.replace('}','').str.replace('{','').str.lower()
addinfo2['MPOG_Case_ID'] = addinfo2['MPOG_Case_ID'].str.replace('-','').str.replace('}','').str.replace('{','').str.lower()

caseinfo1 = caseinfo1.merge(addinfo1[['MPOG_Case_ID', 'MPOG_Patient_ID', 'SMOKE']], on = ['MPOG_Case_ID', 'MPOG_Patient_ID'], how = 'left')
caseinfo1['SMOKE'] = caseinfo1['SMOKE'].fillna(0)

caseinfo2 = caseinfo2.merge(addinfo2[['MPOG_Case_ID', 'MPOG_Patient_ID', 'SMOKE']], on = ['MPOG_Case_ID', 'MPOG_Patient_ID'], how = 'left')
caseinfo2['SMOKE'] = caseinfo2['SMOKE'].fillna(0)

print('Number of entries in Caseinfo1 = {}'.format(len(caseinfo1)))
print('Number of entries in Caseinfo2 = {}'.format(len(caseinfo2)))

print('Number of unique caseID in Caseinfo1 = {}'.format(len(caseinfo1.MPOG_Case_ID.unique())))
print('Number of unique caseID in Caseinfo2 = {}'.format(len(caseinfo2.MPOG_Case_ID.unique())))


Number of entries in Caseinfo1 = 7060060
Number of entries in Caseinfo2 = 4155859
Number of unique caseID in Caseinfo1 = 7060060
Number of unique caseID in Caseinfo2 = 4155859


### Add dialysis info from ICD

In [72]:
#----------- Adding dialysis info from ICD codes (days from case between (-30,30))-----

icd1 = pd.read_csv(os.path.join(data_path, 'PCRC_186_ICDCodes1.txt'), sep = '\t').drop_duplicates()
icd2 = pd.read_csv(os.path.join(data_path, 'PCRC_186_ICDCodes2.txt'), sep = '\t').drop_duplicates()

icd1['DIALYSIS'] = 0
icd1.loc[icd1['Days From Case'] < 0, 'DIALYSIS'] = 1
icd1['MPOG_Patient_ID'] = icd1['MPOG_Patient_ID'].str.replace('-','').str.replace('}','').str.replace('{','').str.lower()
icd1['MPOG_Case_ID'] = icd1['MPOG_Case_ID'].str.replace('-','').str.replace('}','').str.replace('{','').str.lower()
icd1 = icd1.drop_duplicates(subset = 'MPOG_Case_ID', keep = 'first')

icd2['DIALYSIS'] = 0
icd2.loc[icd2['Days From Case'] < 0, 'DIALYSIS'] = 1
icd2['MPOG_Patient_ID'] = icd2['MPOG_Patient_ID'].str.replace('-','').str.replace('}','').str.replace('{','').str.lower()
icd2['MPOG_Case_ID'] = icd2['MPOG_Case_ID'].str.replace('-','').str.replace('}','').str.replace('{','').str.lower()
icd2 = icd2.drop_duplicates(subset = 'MPOG_Case_ID', keep = 'first')

caseinfo1 = caseinfo1.merge(icd1[['MPOG_Case_ID', 'MPOG_Patient_ID', 'DIALYSIS']], on = ['MPOG_Case_ID', 'MPOG_Patient_ID'], how = 'left').drop_duplicates()
caseinfo1['DIALYSIS'] = caseinfo1['DIALYSIS'].fillna(0)

caseinfo2 = caseinfo2.merge(icd2[['MPOG_Case_ID', 'MPOG_Patient_ID', 'DIALYSIS']], on = ['MPOG_Case_ID', 'MPOG_Patient_ID'], how = 'left').drop_duplicates()
caseinfo2['DIALYSIS'] = caseinfo2['DIALYSIS'].fillna(0)

print('Number of entries in Caseinfo1 = {}'.format(len(caseinfo1)))
print('Number of entries in Caseinfo2 = {}'.format(len(caseinfo2)))

print('Number of unique caseID in Caseinfo1 = {}'.format(len(caseinfo1.MPOG_Case_ID.unique())))
print('Number of unique caseID in Caseinfo2 = {}'.format(len(caseinfo2.MPOG_Case_ID.unique())))


Number of entries in Caseinfo1 = 7060060
Number of entries in Caseinfo2 = 4155859
Number of unique caseID in Caseinfo1 = 7060060
Number of unique caseID in Caseinfo2 = 4155859


### Percent transfused table using Primary CPT codes

In [73]:
temp_1 = caseinfo1[['MPOG_Patient_ID', 'Institution', 'NOTHBLEED', 'Anesthesia CPT (Primary)']]
cpt_prim_1 = temp_1.groupby(['Anesthesia CPT (Primary)', 'Institution']).agg({'MPOG_Patient_ID':'count', 'NOTHBLEED': 'sum'}).reset_index(drop = False).rename(columns = {'MPOG_Patient_ID':'count'})
cpt_prim_1['percent_tf'] = (cpt_prim_1['NOTHBLEED']/cpt_prim_1['count'])*100
cpt_prim_1 = cpt_prim_1.loc[cpt_prim_1['count'] >= 50]

cpt_prim_1 = cpt_prim_1.rename(columns = {'NOTHBLEED':'count_transfused', 'percent_tf':'percent_transfused'})

c1_match = caseinfo1.merge(cpt_prim_1, on = ['Anesthesia CPT (Primary)', 'Institution'], how = 'left')
c2_match = caseinfo2.merge(cpt_prim_1, on = ['Anesthesia CPT (Primary)', 'Institution'], how = 'left')

#print(len(c1_match))
#print(c1_match['percent_transfused'].isnull().sum())

#print(len(c2_match))
#print(c2_match['percent_transfused'].isnull().sum())

print('Number of entries in Caseinfo1 = {}'.format(len(caseinfo1)))
print('Number of entries in Caseinfo2 = {}'.format(len(caseinfo2)))
print('Number of entries in c1_match = {}'.format(len(c1_match)))
print('Number of entries in c2_match = {}'.format(len(c2_match)))

print('Number of unique caseIDs in Caseinfo1 = {}'.format(len(caseinfo1)))
print('Number of unique caseIDs in Caseinfo2 = {}'.format(len(caseinfo2)))
print('Number of unique caseIDs in c1_match = {}'.format(len(c1_match)))
print('Number of unique caseIDs in c2_match = {}'.format(len(c2_match)))

Number of entries in Caseinfo1 = 7060060
Number of entries in Caseinfo2 = 4155859
Number of entries in c1_match = 7060060
Number of entries in c2_match = 4155859
Number of unique caseIDs in Caseinfo1 = 7060060
Number of unique caseIDs in Caseinfo2 = 4155859
Number of unique caseIDs in c1_match = 7060060
Number of unique caseIDs in c2_match = 4155859


### Percent transfused table using Predicted CPT codes

In [74]:
#------- Percent tranfused table using Predicted CPT code------------

cpt1 = pd.read_csv(os.path.join(data_path, 'PCRC_186_CPTCodes1.txt'), sep = '\t', encoding ='cp1252')
cpt2 = pd.read_csv(os.path.join(data_path, 'PCRC_186_CPTCodes2.txt'), sep = '\t', encoding ='cp1252')

cpt1['MPOG_Patient_ID'] = cpt1['MPOG_Patient_ID'].str.replace('-','').str.replace('}','').str.replace('{','').str.lower()
cpt1['MPOG_Case_ID'] = cpt1['MPOG_Case_ID'].str.replace('-','').str.replace('}','').str.replace('{','').str.lower()

cpt2['MPOG_Patient_ID'] = cpt2['MPOG_Patient_ID'].str.replace('-','').str.replace('}','').str.replace('{','').str.lower()
cpt2['MPOG_Case_ID'] = cpt2['MPOG_Case_ID'].str.replace('-','').str.replace('}','').str.replace('{','').str.lower()

prior_transfused_1 = caseinfo1.merge(cpt1[['MPOG_Case_ID', 'MPOG_Patient_ID', 'Predicted Anes CPT Code 1']], on = ['MPOG_Case_ID', 'MPOG_Patient_ID'], how = 'inner')
prior_transfused_2 = caseinfo2.merge(cpt2[['MPOG_Case_ID', 'MPOG_Patient_ID', 'Predicted Anes CPT Code 1']], on = ['MPOG_Case_ID', 'MPOG_Patient_ID'], how = 'inner')

cpt_pred_1 = prior_transfused_1[['MPOG_Patient_ID', 'NOTHBLEED', 'Predicted Anes CPT Code 1', 'Institution']].groupby(['Predicted Anes CPT Code 1', 'Institution']).agg({'MPOG_Patient_ID':'count', 'NOTHBLEED': 'sum'}).reset_index(drop = False).rename(columns = {'MPOG_Patient_ID':'count'})
cpt_pred_1['percent_tf'] = (cpt_pred_1['NOTHBLEED']/cpt_pred_1['count'])*100
cpt_pred_1 = cpt_pred_1.loc[cpt_pred_1['count'] >= 50]

cpt_pred_1 = cpt_pred_1.rename(columns = {'NOTHBLEED':'count_transfused', 'percent_tf':'percent_transfused'})

c1_pred_match = prior_transfused_1.merge(cpt_pred_1, on = ['Predicted Anes CPT Code 1', 'Institution'], how = 'left')
c2_pred_match = prior_transfused_2.merge(cpt_pred_1, on = ['Predicted Anes CPT Code 1', 'Institution'], how = 'left')

#print(len(c2_pred_match))
#print(c2_pred_match['percent_transfused'].isnull().sum())

print('Number of entries in Caseinfo1 = {}'.format(len(caseinfo1)))
print('Number of entries in Caseinfo2 = {}'.format(len(caseinfo2)))
print('Number of entries in c1_pred_match = {}'.format(len(c1_pred_match)))
print('Number of entries in c2_pred_match = {}'.format(len(c2_pred_match)))

print('Number of unique caseIDs in Caseinfo1 = {}'.format(len(caseinfo1.MPOG_Case_ID.unique())))
print('Number of unique caseIDs in Caseinfo2 = {}'.format(len(caseinfo2.MPOG_Case_ID)))
print('Number of unique caseIDs in c1_pred_match = {}'.format(len(c1_pred_match)))
print('Number of unique caseIDs in c2_pred_match = {}'.format(len(c2_pred_match)))

# Multiply the percentage transfused by 100

Number of entries in Caseinfo1 = 7060060
Number of entries in Caseinfo2 = 4155859
Number of entries in c1_pred_match = 7060535
Number of entries in c2_pred_match = 4156106
Number of unique caseIDs in Caseinfo1 = 7060060
Number of unique caseIDs in Caseinfo2 = 4155859
Number of unique caseIDs in c1_pred_match = 7060535
Number of unique caseIDs in c2_pred_match = 4156106


### Elective Surgery - defined as not emergency based on ASA score, not on a weekend/holiday, surgery date same as day of admission

In [75]:
c2_match['ELECTSURG'] = 0
c2_match.loc[(c2_match['Emergency Status (ASA Class) Yes/No'] == 'No') & (c2_match['Weekend'] == 'Weekday') & (c2_match['Holiday'] == 'No'), 'ELECTSURG'] = 1

c2_pred_match['ELECTSURG'] = 0
c2_pred_match.loc[(c2_pred_match['Emergency Status (ASA Class) Yes/No'] == 'No') & (c2_pred_match['Weekend'] == 'Weekday') & (c2_pred_match['Holiday'] == 'No'), 'ELECTSURG'] = 1

c1_match['ELECTSURG'] = 0
c1_match.loc[(c1_match['Emergency Status (ASA Class) Yes/No'] == 'No') & (c1_match['Weekend'] == 'Weekday') & (c1_match['Holiday'] == 'No'), 'ELECTSURG'] = 1

c1_pred_match['ELECTSURG'] = 0
c1_pred_match.loc[(c1_pred_match['Emergency Status (ASA Class) Yes/No'] == 'No') & (c1_pred_match['Weekend'] == 'Weekday') & (c1_pred_match['Holiday'] == 'No'), 'ELECTSURG'] = 1


### Adding QM measures (TRAN-01 and TRAN-02)¶

In [76]:
qm1 = pd.read_csv(os.path.join(data_path, 'PCRC_186_QMeasures1.txt'), sep = '\t').drop_duplicates()
qm2 = pd.read_csv(os.path.join(data_path, 'PCRC_186_QMeasures2.txt'), sep = '\t').drop_duplicates()

qm1['MPOG_Case_ID'] = qm1['MPOG_Case_ID'].str.replace('-','').str.replace('}','').str.replace('{','').str.lower()
qm2['MPOG_Case_ID'] = qm2['MPOG_Case_ID'].str.replace('-','').str.replace('}','').str.replace('{','').str.lower()

c1_match = c1_match.merge(qm1, on = ['MPOG_Case_ID'], how = 'left').drop_duplicates()
c1_pred_match = c1_pred_match.merge(qm1, on = ['MPOG_Case_ID'], how = 'left').drop_duplicates()

c2_match = c2_match.merge(qm2, on = ['MPOG_Case_ID'], how = 'left').drop_duplicates()
c2_pred_match = c2_pred_match.merge(qm2, on = ['MPOG_Case_ID'], how = 'left').drop_duplicates()

print('Number of entries in c1_pred_match = {}'.format(len(c1_pred_match)))
print('Number of entries in c2_pred_match = {}'.format(len(c2_pred_match)))
print('Number of entries in c1_match = {}'.format(len(c1_match)))
print('Number of entries in c2_match = {}'.format(len(c2_match)))

print('Number of unique caseIDs in c1_pred_match = {}'.format(len(c1_pred_match.MPOG_Case_ID.unique())))
print('Number of unique caseIDs in c2_pred_match = {}'.format(len(c2_pred_match.MPOG_Case_ID.unique())))
print('Number of unique caseIDs in c1_match = {}'.format(len(c1_match.MPOG_Case_ID.unique())))
print('Number of unique caseIDs in c2_match = {}'.format(len(c2_match.MPOG_Case_ID.unique())))

Number of entries in c1_pred_match = 7060060
Number of entries in c2_pred_match = 4155859
Number of entries in c1_match = 7060060
Number of entries in c2_match = 4155859
Number of unique caseIDs in c1_pred_match = 7060060
Number of unique caseIDs in c2_pred_match = 4155859
Number of unique caseIDs in c1_match = 7060060
Number of unique caseIDs in c2_match = 4155859


### Converting cols to NSQIP/WashU format and saving dataframes

In [77]:
non_model_cols =  ['Medical School Affiliation', 'Hospital Bed Size','Mortality (In Hospital 30-day)', 'ASA Class', 'Anesthesia CPT - Base Unit Value', 'TRAN-01', 'TRAN-02', 'Race']

col_dict_df = pd.read_excel('MPOG_variables_final.xlsx')
col_dict = col_dict_df.set_index('MPOG')['NSQIP/WASHU'].to_dict()

c2_match_reqd = c2_match[[col for col in c2_match.columns if col in col_dict_df['MPOG'].values]].rename(columns = col_dict)
c2_match_reqd[non_model_cols] = c2_match[non_model_cols]
assert len(c2_match_reqd) == c2_match_reqd.CaseID.nunique()

c2_pred_match_reqd = c2_pred_match[[col for col in c2_pred_match.columns if col in col_dict_df['MPOG'].values]].rename(columns = col_dict)
c2_pred_match_reqd[non_model_cols] = c2_pred_match[non_model_cols]
assert len(c2_pred_match_reqd) == c2_pred_match_reqd.CaseID.nunique()

c2_pred_match_reqd.drop(columns = 'CPT_PRIMARY').to_csv("c2_pred_match.csv", index=False)
c2_match_reqd.to_csv("c2_match.csv", index=False)


In [83]:
c2_pred_match_reqd.Institution.value_counts().index.values

array([ 69,  57,  68,  84,  19,  89,   1,  58,  36,  10,  16,   7,  65,
        67,  78, 101,  14,  37,  20,  35,   4,  66,  42,  92,  75, 102,
        96,  23,  83, 118,  76,  86,  47,  53,  56,  13,  88,  40,  70,
        91,  46,  99,  82,  64,  80,  30,  98,  31,   8,  38,  32,  48,
        59,  97, 115,  74,  22,   5], dtype=int64)

## Surgeon identifier stats

In [78]:
surg1 = pd.read_csv(os.path.join(root_path, 'PCRC_186_SurgStaff1_20230621.csv'), header = None).rename(columns = {0:'MPOG_Case_ID',1:'Surg_ID'}).drop_duplicates()
surg2 = pd.read_csv(os.path.join(root_path, 'PCRC_186_SurgStaff2_20230621.csv'), header = None).rename(columns = {0:'MPOG_Case_ID',1:'Surg_ID'}).drop_duplicates()

surg1['MPOG_Case_ID'] = surg1['MPOG_Case_ID'].str.replace('-','').str.lower()
surg2['MPOG_Case_ID'] = surg2['MPOG_Case_ID'].str.replace('-','').str.lower()

merge_surg1 = c1_pred_match.drop(columns = 'percent_transfused').merge(surg1, on = 'MPOG_Case_ID', how = 'inner')
merge_surg2 = c2_pred_match.drop(columns = 'percent_transfused').merge(surg2, on = 'MPOG_Case_ID', how = 'inner')

cpt_surg_1 = merge_surg1[['MPOG_Case_ID', 'NOTHBLEED', 'Predicted Anes CPT Code 1', 'Surg_ID']].groupby(['Predicted Anes CPT Code 1', 'Surg_ID']).agg({'MPOG_Case_ID':'count', 'NOTHBLEED': 'sum'}).reset_index(drop = False).rename(columns = {'MPOG_Case_ID':'case_count', 'NOTHBLEED':'event_count'})
cpt_surg_1['percent_tf'] = (cpt_surg_1['event_count']/cpt_surg_1['case_count'])*100
cpt_surg_1 = cpt_surg_1.loc[cpt_surg_1['case_count'] >= 50]

cpt_surg_1 = cpt_surg_1.rename(columns = {'percent_tf':'percent_transfused'})

c1_pred_match_surg = merge_surg1.merge(cpt_surg_1, on = ['Predicted Anes CPT Code 1', 'Surg_ID'], how = 'left')
c2_pred_match_surg = merge_surg2.merge(cpt_surg_1, on = ['Predicted Anes CPT Code 1', 'Surg_ID'], how = 'left')

# print('Number of entries in Caseinfo1 = {}'.format(len(caseinfo1)))
# print('Number of entries in Caseinfo2 = {}'.format(len(caseinfo2)))
# print('Number of entries in c1_pred_match_surg = {}'.format(len(c1_pred_match_surg)))
# print('Number of entries in c2_pred_match_surg = {}'.format(len(c2_pred_match_surg)))

print('Number of entries in c1_pred_match_surg = {}'.format(len(c1_pred_match_surg)))
print('Number of entries in c2_pred_match_surg = {}'.format(len(c2_pred_match_surg)))

print('Number of unique caseIDs in c1_pred_match_surg = {}'.format(len(c1_pred_match_surg.MPOG_Case_ID.unique())))
print('Number of unique caseIDs in c2_pred_match_surg = {}'.format(len(c2_pred_match_surg.MPOG_Case_ID.unique())))


Number of entries in c1_pred_match_surg = 3383244
Number of entries in c2_pred_match_surg = 3685238
Number of unique caseIDs in c1_pred_match_surg = 3234624
Number of unique caseIDs in c2_pred_match_surg = 3476527


In [79]:
non_model_cols =  ['Surg_ID', 'Medical School Affiliation', 'Hospital Bed Size','Mortality (In Hospital 30-day)', 'ASA Class', 'Anesthesia CPT - Base Unit Value', 'TRAN-01', 'TRAN-02', 'Race']

col_dict_df = pd.read_excel('MPOG_variables_final.xlsx')
col_dict = col_dict_df.set_index('MPOG')['NSQIP/WASHU'].to_dict()

c2_pred_match_reqd_surg = c2_pred_match_surg[[col for col in c2_pred_match_surg.columns if col in col_dict_df['MPOG'].values]].rename(columns = col_dict)
c2_pred_match_reqd_surg[non_model_cols] = c2_pred_match_surg[non_model_cols]

c2_pred_match_reqd_surg.drop(columns = 'CPT_PRIMARY').to_csv("c2_pred_match_surg.csv", index=False)
